In [1]:
import numpy as np
import sympy as sym
import numba
import pydae.build as db

\begin{eqnarray}
\dot \delta &=& \Omega_b \left(\omega - \omega_s\right)\\
\dot \omega &=& 1/(2 H) \left(p_m - p_e - D (\omega - \omega_s) \right)\\
\end{eqnarray}

$$ \sf
\Omega_{b} \left(\sf \omega - \omega_{s}\right)
$$

## System definition 

In [2]:
params_dict = {'X_d':1.81,'X1d':0.3,'T1d0':8.0,
               'X_q':1.76,'X1q':0.65,'T1q0':1.0,
               'R_a':0.003,'X_line': 0.05, 
               'H':6,'D':1.0,
               'Omega_b':2*np.pi*50,'omega_s':1.0,
               'v_0':1.0,'theta_0':0.0,
               'T_g':2.0}


u_ini_dict = {'p_t':0.8,'v_1':1.0}  # for the initialization problem
u_run_dict = {'p_m_ref':0.8,'v_f':1.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['delta','omega','e1q','e1d','p_m']    
y_ini_list = ['v_d','v_q','p_e','i_d','i_q','q_t','theta_1','p_m_ref','v_f'] 
y_run_list = ['v_d','v_q','p_e','i_d','i_q','q_t','theta_1','p_t','v_1'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [6]:
ddelta = Omega_b*(omega - omega_s)
domega = 1/(2*H)*(p_m - p_e - D*(omega - omega_s))
de1q = 1/T1d0*(-e1q - (X_d - X1d)*i_d + v_f)
de1d = 1/T1q0*(-e1d + (X_q - X1q)*i_q)
dp_m = 1/T_g*(p_m_ref - p_m)

g_1 = -v_d + v_1*sin(delta - theta_1) 
g_2 = -v_q + v_1*cos(delta - theta_1) 
g_3 = - p_e + i_d*(v_d + R_a*i_d) + i_q*(v_q + R_a*i_q) 
g_4 = v_q + R_a*i_q + X1d*i_d - e1q
g_5 = v_d + R_a*i_d - X1q*i_q - e1d
g_6 = p_t - (v_1*v_0*sin(theta_1 - theta_0))/X_line
g_7 = q_t + (v_1*v_0*cos(theta_1 - theta_0))/X_line - v_1**2/X_line
g_8 = i_d*v_d + i_q*v_q - p_t
g_9 = i_d*v_q - i_q*v_d - q_t

i_t = (i_d**2+i_q**2)**0.5

h_dict = {'delta':delta,'omega':omega,'e1q':e1q,'e1d':e1d,'p_m':p_m,'p_t':p_t,'q_t':q_t,'v_1':v_1,'theta_1':theta_1,'i_t':i_t}
#h_dict = {,'p_t':p_t,'v_1':v_1}
#h_dict = {'delta':delta,'omega':omega,'e1q':e1q,'e1d':e1d,'p_m':p_m,'p_m_test':p_m_test,'p_t':p_t}
sys = {'name':'smib_4ord_ctrl',
       'params_dict':params_dict,
       'f_list':[ddelta,domega,de1q,de1d,dp_m],
       'g_list':[g_1,g_2,g_3,g_4,g_5,g_6,g_7,g_8,g_9],
       'x_list':x_list,
       'y_ini_list':y_ini_list,
       'y_run_list':y_run_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':h_dict
      }

sys = db.system(sys)
db.sys2num(sys)
                

In [4]:
replacements = {'1d0':"\'_{d0}",'1q0':"\'_{q0}",'1d':"\'_d",'1q':"\'_q",' ref}':'}^\star'}
x_string = '\mathbf x = ' + sym.latex(sys['x'])
f_string = '\mathbf f = ' + sym.latex(sys['f'])
y_run_string = '\mathbf y^{run} = ' + sym.latex(sys['y_run'])
y_ini_string = '\mathbf y^{ini} = ' + sym.latex(sys['y_ini'])
g_string = '\mathbf g = ' + sym.latex(sys['g'])
for item in replacements:
    x_string = x_string.replace(item,replacements[item])
    f_string = f_string.replace(item,replacements[item])
    g_string = g_string.replace(item,replacements[item])
    y_ini_string = y_ini_string.replace(item,replacements[item])
    y_run_string = y_run_string.replace(item,replacements[item])
    

In [5]:
sys['h_dict']

{'delta': delta,
 'omega': omega,
 'e1q': e1q,
 'e1d': e1d,
 'p_m': p_m,
 'p_m_test': p_m_test,
 'p_t': p_t}

In [6]:
print(y_run_string)

\mathbf y^{run} = \left[\begin{matrix}v_{d}\\v_{q}\\p_{e}\\i_{d}\\i_{q}\\q_{t}\\\theta_{1}\\p_{t}\\v_{1}\\p_{m test}\end{matrix}\right]


In [7]:
sys    

{'name': 'smib_4ord_ctrl',
 'params_dict': {'X_d': 1.81,
  'X1d': 0.3,
  'T1d0': 8.0,
  'X_q': 1.76,
  'X1q': 0.65,
  'T1q0': 1.0,
  'R_a': 0.003,
  'X_line': 0.05,
  'H': 6,
  'D': 1.0,
  'Omega_b': 314.1592653589793,
  'omega_s': 1.0,
  'v_0': 1.0,
  'theta_0': 0.0,
  'T_g': 2.0},
 'f_list': [Omega_b*(omega - omega_s),
  (-D*(omega - omega_s) - p_e + p_m)/(2*H),
  (-e1q - i_d*(-X1d + X_d) + v_f)/T1d0,
  (-e1d + i_q*(-X1q + X_q))/T1q0,
  (-p_m + p_m_ref)/T_g],
 'g_list': [v_1*sin(delta - theta_1) - v_d,
  v_1*cos(delta - theta_1) - v_q,
  i_d*(R_a*i_d + v_d) + i_q*(R_a*i_q + v_q) - p_e,
  R_a*i_q + X1d*i_d - e1q + v_q,
  R_a*i_d - X1q*i_q - e1d + v_d,
  p_t + v_0*v_1*sin(theta_0 - theta_1)/X_line,
  q_t + v_0*v_1*cos(theta_0 - theta_1)/X_line - v_1**2/X_line,
  i_d*v_d + i_q*v_q - p_t,
  i_d*v_q - i_q*v_d - q_t,
  -p_m + p_m_test],
 'x_list': ['delta', 'omega', 'e1q', 'e1d', 'p_m'],
 'y_ini_list': ['v_d',
  'v_q',
  'p_e',
  'i_d',
  'i_q',
  'q_t',
  'theta_1',
  'p_m_ref',
  'v_f',
